In [1]:
import pickle

d = pickle.load(open('ml_latest-small_core_10_type_hete.pkl', 'rb'))

In [ ]:
d.keys()

In [3]:
import numpy as np

# unique, counts = np.unique(d['edge_index_nps']['user2item'][0], return_counts=True)

In [ ]:
# np.std(counts)

In [4]:
import pandas as pd

In [5]:
movies = pd.read_csv('../raw/raw_movies.csv', sep=';').fillna('')
ratings = pd.read_csv('../raw/raw_ratings.csv', sep=';')
tagging = pd.read_csv('../raw/raw_tagging.csv', sep=';')

In [6]:
def reindex_df_mlsmall(movies, ratings, tagging):
    """

    Args:
        movies:
        ratings:
        tagging:
        genome_tagging:
        genome_tags:

    Returns:

    """
    # Reindex uid
    unique_uids = np.sort(ratings.uid.unique()).astype(np.int)
    uids = np.arange(unique_uids.shape[0]).astype(np.int)
    raw_uid2uid = {raw_uid: uid for raw_uid, uid in zip(unique_uids, uids)}
    ratings['uid'] = np.array([raw_uid2uid[raw_uid] for raw_uid in ratings.uid], dtype=np.int)
    tagging['uid'] = np.array([raw_uid2uid[raw_uid] for raw_uid in tagging.uid], dtype=np.int)

    # Reindex iid
    unique_iids = np.sort(movies.iid.unique()).astype(np.int)
    iids = np.arange(unique_iids.shape[0]).astype(np.int)
    raw_iid2iid = {raw_iid: iid for raw_iid, iid in zip(unique_iids, iids)}
    movies['iid'] = np.array([raw_iid2iid[raw_iid] for raw_iid in movies.iid], dtype=np.int)
    ratings['iid'] = np.array([raw_iid2iid[raw_iid] for raw_iid in ratings.iid], dtype=np.int)
    tagging['iid'] = np.array([raw_iid2iid[raw_iid] for raw_iid in tagging.iid], dtype=np.int)

    # Create tid
    unique_tags = np.sort(tagging.tag.unique()).astype(np.str)
    tids = np.arange(unique_tags.shape[0]).astype(np.int)
    tags = pd.DataFrame({'tid': tids, 'tag': unique_tags})
    tag2tid = {tag: tid for tag, tid in zip(unique_tags, tids)}
    tagging['tid'] = np.array([tag2tid[tag] for tag in tagging.tag], dtype=np.int)
    tagging = tagging.drop(columns=['tag'])

    return movies, ratings, tagging, tags

num_feat_core = 10
num_core = 10

movies = movies.drop_duplicates()
ratings = ratings.drop_duplicates()
tagging = tagging.drop_duplicates()

movies = movies[movies.iid.isin(ratings.iid.unique())]
ratings = ratings[ratings.iid.isin(movies.iid.unique())]
tagging = tagging[tagging.iid.isin(ratings.iid.unique())]
tagging = tagging[tagging.uid.isin(ratings.uid.unique())]

movie_count = ratings['iid'].value_counts()
movie_count.name = 'movie_count'
ratings = ratings[ratings.join(movie_count, on='iid').movie_count > num_core]

user_count = ratings['uid'].value_counts()
user_count.name = 'user_count'
ratings = ratings[ratings.join(user_count, on='uid').user_count > num_core]

movies = movies[movies.iid.isin(ratings.iid.unique())]
tagging = tagging[tagging.iid.isin(ratings.iid.unique())]
tagging = tagging[tagging.uid.isin(ratings.uid.unique())]

tag_count = tagging['tag'].value_counts()
tag_count.name = 'tag_count'
tagging = tagging[tagging.join(tag_count, on='tag').tag_count > num_feat_core]

years = movies.year.to_numpy()
years[years < 1950] = 1950
movies['year'] = years

years = movies.year.to_numpy().astype(np.int)
min_year = min(years)
max_year = max(years)
num_years = (max_year - min_year) // 10
discretized_years = [min_year + i * 10 for i in range(num_years + 1)]
for i in range(len(discretized_years) - 1):
    years[(discretized_years[i] <= years) & (years < discretized_years[i + 1])] = str(
    discretized_years[i])
    years[years < discretized_years[0]] = discretized_years[0]
    years[years >= discretized_years[-1]] = discretized_years[-1]

movies['year'] = years

movies, ratings, tagging, tags = reindex_df_mlsmall(
                    movies, ratings, tagging)

In [8]:
ratings = ratings.sort_values('timestamp')
# ratings = ratings.sample(frac=1, random_state=42)

In [9]:
tf_size = 3000

for i in range(25):
    # print(i)
    tf = ratings[i*tf_size:(i+1)*tf_size]
    # print(len(tf['uid'].unique()))
    counts = tf['uid'].value_counts().to_numpy()
    # print(f'std: {np.std(counts)} mean: {np.mean(counts)}')
    print(f'{i} {len(tf["uid"].unique())} {round(np.mean(counts), 2)} {round(np.std(counts), 2)}')

0 55 54.55 26.63
1 55 54.55 39.15
2 44 68.18 66.75
3 29 103.45 101.18
4 18 166.67 158.06
5 31 96.77 105.44
6 39 76.92 88.71
7 29 103.45 107.94
8 31 96.77 136.9
9 28 107.14 108.54
10 36 83.33 132.79
11 40 75.0 112.99
12 33 90.91 152.89
13 34 88.24 150.04
14 35 85.71 91.41
15 40 75.0 125.48
16 42 71.43 109.72
17 54 55.56 75.02
18 41 73.17 99.54
19 44 68.18 64.19
20 34 88.24 138.2
21 35 85.71 129.39
22 34 88.24 120.1
23 34 88.24 162.11
24 37 81.08 189.6


In [ ]:
counts = ratings['uid'].value_counts().to_numpy()
print(f'cv: {np.mean(counts)/np.std(counts)}')

In [ ]:
len(ratings['uid'].unique())

In [ ]:
ratings.groupby("uid").count().iid.values